In [1172]:
import gensim
from gensim import corpora
import multiprocessing
from gensim.models import KeyedVectors
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import pkg_resources
import pandas as pd
import numpy as np

In [1173]:
import pandas as pd
from gensim import utils
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from gensim.parsing.preprocessing import preprocess_string
import random
import warnings
warnings.filterwarnings("ignore")

In [1174]:
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [1347]:
df = pd.read_csv('tweets_processed.csv', usecols=['tweets', 'clean_keyword', 'clean_location'])
df2 = pd.read_csv('tweets_update.csv', usecols=['denoised_text', 'lem_tokes', 'split'])
df['lems'] = df2['lem_tokes']
df['lems_split'] = df2['split']
df['dn_text'] = df2['denoised_text']

In [1357]:
df

,tweets,clean_keyword,clean_location,lems,lems_split,dn_text,dn_split
0,our deeds are the reason of this earthquake m...,NaN,NaN,our deeds be the reason of this earthquake may...,"['our', 'deeds', 'be', 'the', 'reason', 'of', ...",our deeds are the reason of this earthquake m...,"[our, deeds, are, the, reason, of, this, earth..."
1,forest fire near la ronge sask canada,NaN,NaN,forest fire near la ronge sask canada,"['forest', 'fire', 'near', 'la', 'ronge', 'sas...",forest fire near la ronge sask canada,"[forest, fire, near, la, ronge, sask, canada]"
2,all residents asked to shelter in place are b...,NaN,NaN,all residents ask to shelter in place be be no...,"['all', 'residents', 'ask', 'to', 'shelter', '...",all residents asked to shelter in place are b...,"[all, residents, asked, to, shelter, in, place..."
3,people receive wildfires evacuation orders in...,NaN,NaN,people receive wildfires evacuation order in c...,"['people', 'receive', 'wildfires', 'evacuation...",people receive wildfires evacuation orders in...,"[people, receive, wildfires, evacuation, order..."
4,just got sent this photo from ruby alaska as ...,NaN,NaN,just get send this photo from ruby alaska as s...,"['just', 'get', 'send', 'this', 'photo', 'from...",just got sent this photo from ruby alaska as ...,"[just, got, sent, this, photo, from, ruby, ala..."
...,...,...,...,...,...,...,...
10871,earthquake safety los angeles safety fasteners...,NaN,NaN,earthquake safety los angeles safety fasteners...,"['earthquake', 'safety', 'los', 'angeles', 'sa...",earthquake safety los angeles safety fasteners...,"[earthquake, safety, los, angeles, safety, fas..."
10872,storm in rhode island worse than last hurrica...,NaN,NaN,storm in rhode island worse than last hurrican...,"['storm', 'in', 'rhode', 'island', 'worse', 't...",storm in rhode island worse than last hurrica...,"[storm, in, rhode, island, worse, than, last, ..."
10873,green line derailment in chicago,NaN,NaN,green line derailment in chicago,"['green', 'line', 'derailment', 'in', 'chicago']",green line derailment in chicago,"[green, line, derailment, in, chicago]"
10874,meg issues hazardous weather outlook hwo,NaN,NaN,meg issue hazardous weather outlook hwo,"['meg', 'issue', 'hazardous', 'weather', 'outl...",meg issues hazardous weather outlook hwo,"[meg, issues, hazardous, weather, outlook, hwo]"


In [1349]:
target_df = pd.read_csv('train.csv', usecols=['target'])
target = list(target_df.target)

In [1350]:
train_df, test_df = df.iloc[:6794, :], df.iloc[6794:, :] 
train_df['target'] = train_df.target
train_df.drop_duplicates(subset=['lems'], keep='last', inplace=True)

In [1356]:
df['dn_split'] = df.dn_text.str.split()

In [1351]:
len(train_df)

6794

In [1334]:
len(test_df)

3263

In [1337]:
tweets = list(train_df.lems)
for x in test_df.lems:
    tweets.append(x) 
len(tweets)

10057

In [1049]:
def kw_loc_column_fix(df):
    df['clean_keyword'] = df.clean_keyword.astype(str)
    df['clean_keyword'] = df.clean_keyword.str.lower()
    df['clean_keyword'] = df.clean_keyword.str.replace(r'\bnan\b', r'')
    df['clean_keyword'] = df.clean_keyword.str.replace(r' ', r'_')
    df['clean_keyword'] = df.clean_keyword.str.strip()
    df['clean_location'] = df.clean_location.astype(str)
    df['clean_location'] = df.clean_location.str.lower()
    df['clean_location'] = df.clean_location.str.strip()
    df['clean_location'] = df.clean_location.str.replace(r'\bnan\b', r'')
    df['clean_location'] = df.clean_location.str.replace(r' ', r'_')
    return df

In [1050]:
train_df = kw_loc_column_fix(train_df)
test_df = kw_loc_column_fix(test_df)

In [1051]:
keyword = pd.get_dummies(train_df.clean_keyword, prefix='keyword')
location = pd.get_dummies(train_df.clean_location, prefix='location')
train_df_secondary = pd.concat([keyword, location], axis=1)

keyword_test = pd.get_dummies(test_df.clean_keyword, prefix='keyword')
location_test = pd.get_dummies(test_df.clean_location, prefix='location')
test_df_secondary = pd.concat([keyword_test, location_test], axis=1)

## extract bigrams

In [362]:
from gensim.models.phrases import Phrases, ENGLISH_CONNECTOR_WORDS
from nltk.corpus import stopwords

In [1380]:

def get_documents(text):
    docs = [row.split() for row in text]
    return docs

doc = get_documents(df.dn_text)

def sentence_to_bi_grams(sentence):
    return ' '.join(phrases[sentence])

phrases = Phrases(doc, min_count=5, threshold=11, connector_words=ENGLISH_CONNECTOR_WORDS)

frozen_phrases = phrases.freeze()

bigrams = []
for row in doc: 
    parsed_sentence = sentence_to_bi_grams(row)
    bigrams.append(parsed_sentence)
    
bigrams

['our deeds are the reason of this earthquake may allah forgive us all',
 'forest_fire near la ronge sask canada',
 'all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected',
 'people receive wildfires evacuation orders in california',
 'just_got sent this photo from ruby alaska as smoke from wildfires pours into a school',
 'rocky_fire update california highway closed in both directions due to lake county fire c afire wildfires',
 'flood disaster heavy_rain causes flash flooding of streets in manitou colorado springs areas',
 'i_am on top of the hill and i can see a fire in the woods',
 'there_is an emergency evacuation happening now in the building across the street',
 'i_am afraid that the tornado is coming to our area',
 'three_people died from the heat_wave so_far',
 'haha south tampa is getting flooded hah wait a second i live in south tampa what am i gonna do what am i gonna do fuck flooding',
 'raining f

In [1378]:
df['bigrams'] = bigrams

## prep text for vectorization

In [445]:
import re

import nltk
from nltk.corpus import stopwords
stopword_list = stopwords.words('english')
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if word not in stopword_list and len(word) > 1:
                tokens.append(word.lower())
    return tokens

train_tweets = train_df.stems.astype(str).apply(tokenize_text)
test_tweets = test_df.stems.apply(tokenize_text)

In [1052]:
print(len(train_df))
print(len(test_df))

6830
3263


In [1382]:
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word)>1:
                tokens.append(word.lower())
    return tokens

texts = df.bigrams.apply(tokenize_text)
texts

0        [our, deeds, are, the, reason, of, this, earth...
1             [forest_fire, near, la, ronge, sask, canada]
2        [all, residents, asked, to, shelter, in, place...
3        [people, receive, wildfires, evacuation, order...
4        [just_got, sent, this, photo, from, ruby, alas...
                               ...                        
10871    [earthquake, safety, los_angeles, safety, fast...
10872    [storm, in, rhode, island, worse, than, last, ...
10873                [green_line, derailment, in, chicago]
10874        [meg, issues_hazardous, weather_outlook, hwo]
10875    [cityof, calgary, has, activated, it_is, munic...
Name: bigrams, Length: 10876, dtype: object

In [1383]:
df['texts'] = texts

In [1384]:
train, test = df.iloc[:6794, :], df.iloc[6794:, :] 
train['target'] = train_df.target

In [1385]:
X_train, X_test, y_train, y_test = train_test_split(train.texts, train.target, test_size=0.1, random_state=42)

In [1386]:
from collections import Counter
from statistics import mean

word_dict = Counter(X_train.sum())
# count number of words in corpus
num_words = sum(word_dict[w] for w in word_dict)
print(f'There are {num_words} words in the training dataset')

# count number of unique words in corpus
word_count_sorted = [(value, key) for key, value in word_dict.items()]
word_count_sorted.sort()
vocab_size = len(word_count_sorted)
print(f'There are {vocab_size} unique words in the training dataset')
print("max len of training tweets",max([len(x) for x in X_train]))
print("max len of test tweets",max([len(x) for x in X_test]))
print("min len of training tweets",min([len(x) for x in X_train]))
print("min len of test tweets",min([len(x) for x in X_test]))
print("mean len of training tweets", mean([len(x) for x in X_train]))
print("mean len of test tweets",mean([len(x) for x in X_test]))

There are 71421 words in the training dataset
There are 13337 unique words in the training dataset
max len of training tweets 29
max len of test tweets 27
min len of training tweets 1
min len of test tweets 1
mean len of training tweets 11.681550539744848
mean len of test tweets 11.80735294117647


In [1387]:
def get_embedding(word, model, vector_size):
    if word in model.wv.vocab:
        return model[x]
    else: 
        return np.zeros(vector_size)

def get_average_vector(model, list_of_tweets):    
    vec_dicts = {}
    for tweet in list_of_tweets:
        average_vector = (np.mean(np.array([get_embedding(tweet, model)]), axis=0))
        v_dict = {tweet : (average_vector)}
        vec_dicts.update(v_dict)
        return vec_dicts

In [1388]:
from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction
import numpy as np     
import plotly

In [1389]:
def reduce_dimensions(model):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    # extract the words & their vectors, as numpy arrays
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  # fixed-width numpy strings

    # reduce using t-SNE
    tsne = TSNE(n_components=num_dimensions, random_state=42)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels


def plot_with_plotly(x_vals, y_vals, labels, plot_in_notebook=True):
    from plotly.offline import init_notebook_mode, iplot, plot
    import plotly.graph_objs as go

    trace = go.Scatter(x=x_vals, y=y_vals, mode='text', text=labels)
    data = [trace]

    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')


def plot_with_matplotlib(x_vals, y_vals, labels):
    import matplotlib.pyplot as plt
    import random

    random.seed(0)

    plt.figure(figsize=(12, 12))
    plt.scatter(x_vals, y_vals)

    #
    # Label randomly subsampled 25 data points
    #
    indices = list(range(len(labels)))
    selected_indices = random.sample(indices, 25)
    for i in selected_indices:
        plt.annotate(labels[i], (x_vals[i], y_vals[i]))

try:
    get_ipython()
except Exception:
    plot_function = plot_with_matplotlib
else:
    plot_function = plot_with_plotly



## uninitialized skipgram

In [1390]:
cores = multiprocessing.cpu_count()

In [1402]:
# uninitializeed model
model_ui_sg = Word2Vec(
        vector_size=300,
        sg=1,

        min_count=4,
        window=2,
        workers=cores)
vocab = model_ui_sg.build_vocab(X_train)

model_ui_sg.train(X_train, total_examples=len(X_train), total_words=num_words, compute_loss=True, epochs=10)

(487828, 714210)

In [1403]:
model_ui_sg.wv.most_similar(positive="earthquake")

[('august', 0.9988714456558228),
 ('reports', 0.9987934231758118),
 ('demolition', 0.9987364411354065),
 ('outbreak', 0.9987363219261169),
 ('derailment', 0.9987184405326843),
 ('colorado', 0.998711109161377),
 ('pakistan', 0.9987068772315979),
 ('july', 0.9986609816551208),
 ('region', 0.9986537098884583),
 ('western', 0.9986401796340942)]

In [1404]:
x_vals, y_vals, labels = reduce_dimensions(model_ui_sg)
plot_function(x_vals, y_vals, labels)

## self initialized skipgram

In [1420]:
# uninitializeed model
model_si_sg = Word2Vec(
        sentences=X_train,
        vector_size=100,
        sg=0,
        min_count=4,
        window=3,
        workers=cores)
vocab = model_si_sg.build_vocab(X_train)

model_si_sg.train(X_train, total_examples=len(X_train), total_words=num_words, compute_loss=True, epochs=20)

(975375, 1428420)

In [1421]:
model_si_sg.wv.most_similar(positive="earthquake")

[('pakistan', 0.994979739189148),
 ('central', 0.9944935441017151),
 ('reports', 0.9940882325172424),
 ('africa', 0.9934355616569519),
 ('legionnaires', 0.993368923664093),
 ('derailment', 0.9933409690856934),
 ('libya', 0.9928828477859497),
 ('area', 0.9928755164146423),
 ('myanmar', 0.9927868843078613),
 ('coordinated_universal', 0.9927593469619751)]

In [1422]:
x_vals, y_vals, labels = reduce_dimensions(model_si_sg)
plot_function(x_vals, y_vals, labels)

# cbow

In [1412]:
# uninitializeed model
model_ui_bow = Word2Vec(
        vector_size=100,
        sg=0,
        #negative=5,
        #ns_exponent=0.3,
        cbow_mean=1,
        sample=1e-3, 
        min_count=4,
        window=3, 
        workers=cores)
vocab = model_ui_bow.build_vocab(X_train)

model_ui_bow.train(X_train, total_examples=model_ui_bow.corpus_count, epochs=30)

(1463181, 2142630)

In [1413]:
model_ui_bow.wv.most_similar(positive="wildfire")

[('rocky_fire', 0.9788936972618103),
 ('northern_california', 0.9774787425994873),
 ('wildfire_abc', 0.9707353711128235),
 ('homes_razed', 0.9680832624435425),
 ('experiments', 0.952380895614624),
 ('damaged', 0.9485912322998047),
 ('detonate_feat', 0.9477091431617737),
 ('latest_more', 0.947328507900238),
 ('apollo_brown', 0.9455264806747437),
 ('m_o', 0.9437326192855835)]

In [1414]:
model_ui_bow.get_latest_training_loss()

0.0

In [1415]:
x_vals, y_vals, labels = reduce_dimensions(model_ui_bow)
plot_function(x_vals, y_vals, labels)

In [546]:
##### model.save("w2v_ui.model")
model = Word2Vec.load("w2v_ui.model")
word_vectors = model_ui.wv
word_vectors.save("w2v_ui.wordvectors")

NameError: name 'model_ui' is not defined

In [1423]:
# self initialized model
model_si_bow = Word2Vec(
        X_train,
        vector_size=50, 
        #sample=1e-5,
        min_count=5,
        window=2, 
        workers=cores)
vocab = model_si_bow.build_vocab(X_train)

model_si_bow.train(X_train,  total_examples=len(X_train), total_words=num_words, epochs=10)

(454388, 714210)

In [1424]:
model_si_bow.wv.most_similar(positive="wildfire")

[('wildfire_abc', 0.995316207408905),
 ('armageddon', 0.9953052401542664),
 ('several', 0.9951783418655396),
 ('party', 0.9951189756393433),
 ('latest', 0.9950906038284302),
 ('rain', 0.9950850009918213),
 ('radio', 0.9950726628303528),
 ('municipal_emergency', 0.9949969053268433),
 ('turkish', 0.9949257373809814),
 ('ur', 0.9948827624320984)]

In [1425]:
x_vals, y_vals, labels = reduce_dimensions(model_si_bow)
plot_function(x_vals, y_vals, labels)

## pretrained word_vectors model

In [1126]:
from nltk.tokenize import word_tokenize
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
import gensim
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [1432]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

glove_file = datapath("C:\\Users\\root\\Projects\\nlp\\disaster_tweets\\utils\\glove.6B.100d.txt")
tmp_file = get_tmpfile("C:\\Users\\root\\Projects\\nlp\\disaster_tweets\\utils\\test_word2vec.txt")

_ = glove2word2vec(glove_file, tmp_file)

model = KeyedVectors.load_word2vec_format(tmp_file, binary=False)

In [1435]:
words = list(model.wv)
#model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary = True) 
model.vector_size

AttributeError: 'KeyedVectors' object has no attribute 'wv'

In [1428]:
# Filter the list of vectors to include only those that Word2Vec has a vector for
vector_list = [model[word] for word in words if word in model.vocab]

# Create a list of the words corresponding to these vectors
words_filtered = [word for word in words if word in model.vocab]

# Zip the words together with their vector representations
word_vec_zip = zip(words_filtered, vector_list)

# Cast to a dict so we can turn it into a DataFrame
word_vec_dict = dict(word_vec_zip)
df = pd.DataFrame.from_dict(word_vec_dict, orient='index')
df.head(3)
#model.train(X_train, total_examples=model.corpus_count, epochs=30)

AttributeError: 'KeyedVectors' object has no attribute 'train'

In [1302]:

# load the whole embedding into memory
embeddings_index = dict()
f = open('glove.6B.100d.txt', encoding="utf8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [1359]:
from gensim.corpora import Dictionary

corpus = [df.dn_text]
dct = Dictionary(corpus)
len(dct)

9581

In [1303]:
token = Tokenizer()
token.fit_on_texts(texts)

X = token.texts_to_sequences(texts)
print(X[:5])

[[61, 5181, 6, 1, 609, 4, 19, 467, 3626, 1884, 3627], [447, 176, 2213, 7210, 7211, 758], [40, 1433, 353, 2, 1631, 3, 354, 6, 6, 7212, 11, 1260, 37, 322, 323, 27, 1631, 3, 354, 653, 2767], [43, 2768, 946, 1135, 3, 139], [947, 368, 19, 272, 12, 5182, 1136, 30, 159, 12, 946, 2769, 5183]]


In [1426]:
c = token.texts_to_matrix(texts, mode='tfidf')

In [1427]:
c[0].shape

(13994,)

In [1306]:
token.word_index

{'the': 1,
 'to': 2,
 'in': 3,
 'of': 4,
 'and': 5,
 'be': 6,
 'on': 7,
 'for': 8,
 'with': 9,
 'at': 10,
 'by': 11,
 'from': 12,
 'my': 13,
 'have': 14,
 'that': 15,
 'it_be': 16,
 'you': 17,
 'an': 18,
 'this': 19,
 'do_not': 20,
 'not': 21,
 'your': 22,
 'get': 23,
 'me': 24,
 'it': 25,
 'via': 26,
 'or': 27,
 'go': 28,
 'like': 29,
 'as': 30,
 'fire': 31,
 'up': 32,
 'but': 33,
 'just': 34,
 'out': 35,
 'so': 36,
 'no': 37,
 'after': 38,
 'news': 39,
 'all': 40,
 'new': 41,
 'do': 42,
 'people': 43,
 'time': 44,
 'one': 45,
 'her': 46,
 'when': 47,
 'about': 48,
 'us': 49,
 'flood': 50,
 'his': 51,
 'say': 52,
 'now': 53,
 'there_be': 54,
 'will': 55,
 'how': 56,
 'some': 57,
 'bomb': 58,
 'them': 59,
 'back': 60,
 'our': 61,
 'retweet': 62,
 'over': 63,
 'make': 64,
 'day': 65,
 'be_not': 66,
 'see': 67,
 'love': 68,
 'if': 69,
 'police': 70,
 'come': 71,
 'loud': 72,
 'can_not': 73,
 'that_be': 74,
 'work': 75,
 'live': 76,
 'man': 77,
 'think': 78,
 'attack': 79,
 'scream': 80,


In [1307]:
vocab_size = len(token.word_index)
embeddings_dict = token.word_index

In [1308]:
len(c)

6794

In [1309]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size+1, 100))

for word, i in token.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [1310]:
def find_closest_embeddings(embedding):
    return sorted(embeddings_index.keys(), key=lambda word: spatial.distance.euclidean(embeddings_index[word], embedding))

In [1311]:
find_closest_embeddings(embeddings_index["earthquake"])

['earthquake',
 'quake',
 'tsunami',
 'temblor',
 'magnitude',
 'disaster',
 'aftershock',
 'earthquakes',
 'tremor',
 'quakes',
 'tsunamis',
 'jolted',
 'devastated',
 'devastating',
 'epicenter',
 'aftershocks',
 'disasters',
 'sumatra',
 'devastation',
 'richter',
 'floods',
 'struck',
 'tremors',
 'katrina',
 'flood',
 'flooding',
 'storm',
 'damage',
 'undersea',
 'measuring',
 'mudslide',
 'cyclone',
 'rocked',
 'aftermath',
 'jolts',
 'rattled',
 'magnitude-6',
 'catastrophe',
 'affected',
 'sichuan',
 'scale',
 'stricken',
 'explosion',
 'typhoon',
 'jolt',
 'calamity',
 'triggered',
 'epicentre',
 'blast',
 'ravaged',
 'wenchuan',
 'tragedy',
 'catastrophic',
 'shook',
 'storms',
 'occurred',
 'morakot',
 '7.6',
 'waves',
 'seismologists',
 '6.8',
 'seismic',
 '7.2',
 '7.2-magnitude',
 'hurricane',
 'magnitude-5',
 'tidal',
 'magnitude-7',
 'damaged',
 'tornado',
 '6.2',
 'spill',
 'temblors',
 'massive',
 'tangshan',
 '7.6-magnitude',
 'rainstorm',
 'landslides',
 'rattles',


In [1429]:

tsne = TSNE(n_components=2, random_state=0)
words =  list(embeddings_dict.values())
for word in words:
    vectors = [embeddings_dict[word] for word in words]
Y = tsne.fit_transform(vectors[:1000])
plt.scatter(Y[:, 0], Y[:, 1])

for label, x, y in zip(words, Y[:, 0], Y[:, 1]):
    plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords="offset points")
plt.show()

KeyError: 1

In [1140]:
import gensim.downloader
# Show all available models in gensim-data
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [1141]:
# pretrained model
model_pt = Word2Vec(glove_vectors,
                     window=3,
                     vector_size=50,
                     alpha=0.1, 
                     min_alpha=0.001, 
                     workers=cores-1)
vocab = model_pt.build_vocab(texts)

model_pt.train(X_train, total_examples=len(X_train), epochs=10)

NameError: name 'glove_vectors' is not defined

In [ ]:
model_pt.wv.most_similar(positive="earthquake")

pretrained_w2v_model.save("w2v_pt.model")
model_pt = Word2Vec.load("w2v_pt.model")
word_vectors_pt = model_pt.wv
word_vectors_pt.save("w2v_pt_wv.wordvectors")

In [ ]:
model_pt.train(train_X, total_examples=pretrained_w2v_model.corpus_count, epochs=30, report_delay=1)

In [ ]:
wv = KeyedVectors.load("w2v_pt_wv.wordvectors", mmap='r')

In [ ]:
wv

In [1166]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline

from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.svm import SVC

In [1167]:

# the models that you want to compare
models = {

    'clf_knn': KNeighborsClassifier(),
    'clf_svm': SVC(),
    #'clf_xgb': XGBClassifier(),
    #'clf_ada': AdaBoostClassifier(),
 
}

# the optimisation parameters for each of the above models
params = {
#     'clf_randforest':{ 
#         'n_jobs': [-1],
#         'max_features': ['sqrt'],
#         'criterion': ['entropy'],
#         'penalty': [10, 25, 36, 64],
       
#          },
    
    'clf_knn': {
        'weights': [ 'distance'],
        'algorithm': ['ball_tree', 'kd_tree']
         },
    

    

    'clf_svm' : {
        'kernel': ['rbf', 'sigmoid'], 
     
        
         'C': [.01, .1, 1, 10, 100, 1000],
          },
    
#     'clf_xgb' : {
#         'objective': ['binary:logistic'],
#         'learning_rate': [0.001, 0.05, 0.1],
#         'alpha': [0.001, 0.3, 0.05]
#         },
        

                   
                 }

In [ ]:
def fit_search(X_data, y_data):
        """
        fits the list of models to the training data, thereby obtaining in each 
        case an evaluation score after GridSearchCV cross-validation
        """
        for name in models.keys():
            est = models[name]
            est_params = params[name]
            gscv = GridSearchCV(estimator=est, param_grid=est_params, cv=10)
            gscv.fit(X_data, y_data)
            print("best parameters are: {}".format(gscv.best_estimator_))
            y_pred = gscv.predict(X_data)
            print(classification_report(y_data, y_pred))

In [ ]:

import warnings
warnings.filterwarnings('ignore') 
fit_search(X_train, y_train)

In [ ]:
logreg = LogisticRegression(n_jobs=1, C=1e3)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
from sklearn.metrics import accuracy_score, f1_score
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

In [ ]:
y_train, X_train = vec_for_learning(model_dmm, train_tagged)
y_test, X_test = vec_for_learning(model_dmm, test_tagged)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

In [ ]:
tweets_df

## train, test = train_test_split(train_df, random_state=42, test_size=0.2)

In [1099]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from collections import Counter

In [1092]:
# verify shape of data
print('Test label shape:', np.shape(y_test))
print('Train label shape:', np.shape(y_train))

print('Test shape:', np.shape(X_test))
print('Train shape:', np.shape(X_train))

Test label shape: (683,)
Train label shape: (6147,)
Test shape: (683,)
Train shape: (6147,)


In [1120]:
from nltk.corpus import stopwords
def tokenize_text_sw(text):
    tokens = []
    stopword_list =  list(stopwords.words('english'))
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word)>1 :
                tokens.append(word.lower())
    return tokens

texts = train_df.tweets.astype(str).apply(tokenize_text)
texts

0       [our, deeds, are, the, reason, of, this, earth...
1           [forest, fire, near, la, ronge, sask, canada]
2       [all, residents, asked, to, shelter, in, place...
3       [people, receive, wildfires, evacuation, order...
4       [just, got, sent, this, photo, from, ruby, ala...
                              ...                        
7608    [two, giant, cranes, holding, bridge, collapse...
7609    [the, out, of, control, wild, fires, in, calif...
7610                       [utc, km, of, volcano, hawaii]
7611    [police, investigating, after, an, bike, colli...
7612    [the, latest, more, homes, razed, by, northern...
Name: tweets, Length: 6830, dtype: object

In [1121]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split

In [1125]:
train_X = str(texts)
train_y = X_test
# convert target data with labelencoder
train_y= train_y.astype('category')
le = LabelEncoder()   
y = np.array(train_y)

le.fit_transform(y)
y = y.ravel()
# define independent variable
X = train_X
#X = X.reshape(-1,1)
# split data 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42,
                                                     test_size=.1, stratify=y)


TypeError: unhashable type: 'list'

In [1123]:
#(ngram_range=(1,2)
pipeline = Pipeline([ ('vect', CountVectorizer(ngram_range=(1,2))), 
                     ('tfidf', TfidfTransformer(use_idf=True))])
X_train = pipeline.fit_transform(X_train)
X_test = pipeline.transform(X_test)



counter = Counter(y_train)
print(counter)

#assign variable to np.shape tuple, width dimension
input_dimension = np.shape(X_train)
count_terms = input_dimension[1]
print(count_terms)

AttributeError: 'list' object has no attribute 'lower'

In [1151]:
vocab_size = 10000  # Only consider the top 20k words
num_tokens_per_example = 15  # Only consider the first 200 words of each movie review
X_train = keras.preprocessing.sequence.pad_sequences(
    X_train, maxlen=num_tokens_per_example
)
x_val = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=num_tokens_per_example) 

ValueError: invalid literal for int() with base 10: 'coming'

In [1150]:
from tensorflow import keras
